<a href="https://colab.research.google.com/github/ndr-bdnt/hacktiv8-session4-submission/blob/main/ticker_summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance


In [12]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta


In [16]:
TICKERS = [
    "COAL.JK",
    "ADRO.JK",
    "PTBA.JK",
    "BBCA.JK",
    "TLKM.JK"
]
TODAY = datetime.today()
START_DATE = (TODAY - timedelta(weeks=24)).strftime("%Y-%m-%d")

results = []

# =========================================================
# LOOP PER TICKER
# =========================================================
for ticker in TICKERS:
    print(f"\nProcessing {ticker} ...")

    try:
        # --- DAILY DATA ---
        df_d1 = yf.download(
            ticker,
            start=START_DATE,
            interval="1d",
            auto_adjust=False,
            progress=False
        )

        if df_d1.empty:
            print("  ⚠️ No daily data, skipped")
            continue

        df_d1.columns = df_d1.columns.get_level_values(0)
        df_d1.index = pd.to_datetime(df_d1.index)
        df_d1 = df_d1.sort_index().dropna()

        # --- WEEKLY DATA ---
        df_w1 = df_d1.resample("W-FRI").agg({
            "Open": "first",
            "High": "max",
            "Low": "min",
            "Close": "last",
            "Volume": "sum"
        }).dropna()

        # --- STRUCTURAL FACTS ---
        latest_close = df_d1["Close"].iloc[-1]
        weekly_high = df_w1["High"].max()
        weekly_low = df_w1["Low"].min()

        dist_high = (latest_close - weekly_high) / weekly_high * 100
        dist_low = (latest_close - weekly_low) / weekly_low * 100

        # --- SAVE RESULT ---
        results.append({
            "Ticker": ticker,
            "Latest Close": round(latest_close, 2),
            "Weekly High": round(weekly_high, 2),
            "Weekly Low": round(weekly_low, 2),
            "Distance from High (%)": round(dist_high, 2),
            "Distance from Low (%)": round(dist_low, 2),
            "Daily Bars": len(df_d1),
            "Weekly Bars": len(df_w1)
        })

    except Exception as e:
        print(f"  ❌ Error processing {ticker}: {e}")

# =========================================================
# FINAL RESULT TABLE
# =========================================================
df_result = pd.DataFrame(results)

print("\n=== SUMMARY ===")
df_result


Processing COAL.JK ...

Processing ADRO.JK ...

Processing PTBA.JK ...

Processing BBCA.JK ...

Processing TLKM.JK ...

=== SUMMARY ===


,Ticker,Latest Close,Weekly High,Weekly Low,Distance from High (%),Distance from Low (%),Daily Bars,Weekly Bars
0,COAL.JK,89.0,148.0,31.0,-39.86,187.10,115,25
1,ADRO.JK,1820.0,2050.0,1625.0,-11.22,12.00,115,25
2,PTBA.JK,2320.0,2500.0,2170.0,-7.20,6.91,115,25
3,BBCA.JK,8025.0,8975.0,7225.0,-10.58,11.07,115,25
4,TLKM.JK,3470.0,3720.0,2750.0,-6.72,26.18,115,25
